In [1]:
import ast
import datetime
import json
import os
import time

from elasticsearch import Elasticsearch
import numpy as np
import pandas as pd
import pytrec_eval
import torch
from tqdm import tqdm

from query_functions import query_elasticsearch_hybrid

current_timestamp = datetime.datetime.now()


In [2]:
# Connect to local elastic

es = Elasticsearch('http://localhost:9200')
es.ping()

True

# Load Data

In [3]:
def convert_to_dict(string):
    try:
        return ast.literal_eval(string)
    except (SyntaxError, ValueError):
        return None  # Handle cases where the string is not a valid dictionary representation


In [4]:
def dict_to_string(my_dict):
  result_str = ""
  for key, value in my_dict.items():
    result_str += str(key) + ' ' + str(value) + ' '
  return result_str

In [5]:
# Products

filename = "processed_data/df_prods.csv"

if os.path.isfile(filename):
    df_prods = pd.read_csv(filename)
    df_prods['product_attributes'] = df_prods['product_attributes'].apply(convert_to_dict)

else:
    print('Cannot locate file')

df_prods.head()

,product_uid,product_title,product_description,product_attributes
0,100001,Simpson Strong-Tie 12-Gauge Angle,"Not only do angles make joints stronger, they ...",{'Bullet01': 'Versatile connector for various ...
1,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,BEHR Premium Textured DECKOVER is an innovativ...,"{'Application Method': 'Brush,Roller,Spray', '..."
2,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,Update your bathroom with the Delta Vero Singl...,"{'Bath Faucet Type': 'Combo Tub and Shower', '..."
3,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,Achieving delicious results is almost effortle...,"{'Appliance Type': 'Over the Range Microwave',..."
4,100007,Lithonia Lighting Quantum 2-Light Black LED Em...,The Quantum Adjustable 2-Light LED Black Emerg...,"{'Battery Power Type': 'Ni-Cad', 'Battery Size..."


In [6]:
df_prods['product_attributes_string'] = [dict_to_string(x) if x is not None else x for x in df_prods['product_attributes']]
df_prods['product_text_string'] = df_prods['product_title'].fillna('') + ' ' + df_prods['product_description'].fillna('') + ' ' + df_prods['product_attributes_string'].fillna('') 

In [7]:
pd.Series([len(x) for x in df_prods['product_text_string']]).describe()

count    124603.000000
mean       1504.848262
std         904.592258
min         123.000000
25%         888.000000
50%        1297.000000
75%        1890.000000
max       11606.000000
dtype: float64

In [8]:
# Query

filename = "processed_data/df_queries.csv"

if os.path.isfile(filename):
    df_queries = pd.read_csv(filename)
    
else:
    print('Cannot locate file')

df_queries.head()

,search_term,query_id,has_relevant_results
0,angle bracket,6e0a07626e48aee6f7ce9ec6cd753426d6acafded1598f...,1
1,l bracket,5863e75dfdc9ae5db3f6b4dbddf129d5568e085bf57711...,1
2,deck over,406b3569b2db043604fdb42a67f4ec49964a5ff07cddf0...,1
3,rain shower head,49b2dc56a0e1945c435c1579c07df519878619e3e8d59d...,1
4,shower only faucet,7620551bacb6cdddca5f33ec0943cea7971095a1e9be06...,1


In [9]:
# Relevance

filename = "processed_data/df_relevance.csv"

if os.path.isfile(filename):
    df_relevance = pd.read_csv(filename)
    
else:
    print('Cannot locate file')

df_relevance.head()

,query_id,product_uid,relevance
0,6e0a07626e48aee6f7ce9ec6cd753426d6acafded1598f...,100001,3.00
1,5863e75dfdc9ae5db3f6b4dbddf129d5568e085bf57711...,100001,2.50
2,406b3569b2db043604fdb42a67f4ec49964a5ff07cddf0...,100002,3.00
3,49b2dc56a0e1945c435c1579c07df519878619e3e8d59d...,100005,2.33
4,7620551bacb6cdddca5f33ec0943cea7971095a1e9be06...,100005,2.67


# Create Embeddings

In [10]:
from sentence_transformers import SentenceTransformer

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model = SentenceTransformer("Snowflake/snowflake-arctic-embed-m", trust_remote_code=True)

print(model)

c:\Users\Billy\miniforge3\envs\homedepot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cpu device


You try to use a model that was created with version 2.7.0.dev0, however, your version is 2.6.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


In [11]:
# Embeddings were run on Kaggle for access GPU

# df_prods_docs = df_prods['product_text_string'].tolist()
# prod_embeddings = model.encode(df_prods_docs)
# np.save('/kaggle/working/prod_embeddings.npy', prod_embeddings)

# prompt = 'Represent this sentence for searching relevant passages: '
# df_queries_docs = [prompt + str(x) for x in df_queries['search_term']]
# query_embeddings = model.encode(df_queries_docs)
# np.save('/kaggle/working/query_embeddings.npy', query_embeddings)

prod_embeddings = np.load('processed_data/prod_embeddings.npy')
query_embeddings = np.load('processed_data/query_embeddings.npy')

In [12]:
df_queries['query_embedding'] = [embedding.tolist() for embedding in query_embeddings]
df_queries.head()

,search_term,query_id,has_relevant_results,query_embedding
0,angle bracket,6e0a07626e48aee6f7ce9ec6cd753426d6acafded1598f...,1,"[-0.058361075818538666, 0.026495283469557762, ..."
1,l bracket,5863e75dfdc9ae5db3f6b4dbddf129d5568e085bf57711...,1,"[-0.027898991480469704, -0.024725843220949173,..."
2,deck over,406b3569b2db043604fdb42a67f4ec49964a5ff07cddf0...,1,"[-0.07859756052494049, -0.00036610415554605424..."
3,rain shower head,49b2dc56a0e1945c435c1579c07df519878619e3e8d59d...,1,"[-0.020222697407007217, 0.06791018694639206, 0..."
4,shower only faucet,7620551bacb6cdddca5f33ec0943cea7971095a1e9be06...,1,"[-0.003950382117182016, 0.012035505846142769, ..."


# Load Into Index

In [13]:
# Initialize the index

index_name = 'products-embeddings'

mapping = {
    "properties": {
        "product_uid": {
            "type": "integer"
        },
        "product_title": {
            "type": "text"
        },
        "product_description": {
            "type": "text"
        },
        "product_attributes": {
            "type": "nested",
            "properties": {
                "name": {
                    "type": "text"
                },
                "value": {
                    "type": "text"
                },
                "name_value": {
                    "type": "text"
                },
            }
        },
        "product_vector": {
            "type": "dense_vector",
            "dims": 768
        },
        "query_scores": {
            "type": "nested",
            "properties": {
                "query_id": {
                    "type": "text"
                },
                "relevance": {
                    "type": "float"
                },
            }
        }
    }
}

#es.indices.create(index=index_name, mappings=mapping)

In [ ]:
# Create documents for indexing

product_document_list = []

for index, row in tqdm(df_prods.iterrows(), total=len(df_prods)):
    
    query_scores = []
    tmp_query = df_relevance[(df_relevance['product_uid']==row['product_uid'])]
    if len(tmp_query)>0:
        tmp_query = tmp_query.replace(pd.NA, '', regex=True)
        for index_q, row_q in tmp_query.iterrows():
            query_scores.append({'query_id': row_q['query_id'],
                                 'relevance': row_q['relevance']})

    product_attributes = []
    tmp_attr = row['product_attributes']
    if not pd.isnull(tmp_attr):
        for k in tmp_attr.keys():
            product_attributes.append({'name': k,
                                    'value': tmp_attr[k],
                                    'name_value': str(k) + ' ' + str(tmp_attr[k])})

    tmp_doc = {
        'product_uid': row['product_uid'],
        'product_title': row['product_title'],
        'product_description': row['product_description'],
        'product_attributes': product_attributes,
        'product_vector': list(prod_embeddings[index]),
        'query_scores': query_scores
    }
    product_document_list.append(tmp_doc)

In [ ]:
# Index the documents

for p in tqdm(product_document_list):
    es.index(index=index_name, document=p)

# KNN Queries

## Sample Query

In [14]:
# A sample query

search_vector = df_queries['query_embedding'][0]

results = query_elasticsearch_hybrid(es, index_name, search_vector=search_vector, num_results=10)

hits = pd.DataFrame(results['hits']['hits'])

In [15]:

product_uids = []
product_titles = []
product_descriptions = []
product_attributes = []
query_id_list = []
relevances = []
for h in hits['_source']:
    product_uids.append(h['product_uid'])
    product_titles.append(h['product_title'])
    product_descriptions.append(h['product_description'])
    product_attributes.append(h['product_attributes'])
    
    if len(h['query_scores'])>0:
        query_id_list.append(h['query_scores'][0]['query_id'])
        relevances.append(h['query_scores'][0]['relevance'])
    else:
        query_id_list.append(None)
        relevances.append(None)

hits['product_uid'] = product_uids
hits['product_title'] = product_titles
hits['product_description'] = product_descriptions
hits['product_attribute'] = product_attributes
hits['query_id'] = query_id_list
hits['relevance'] = relevances

hits.head()


,_index,_id,_score,_source,product_uid,product_title,product_description,product_attribute,query_id,relevance
0,products-embeddings,3JIUFo8B20Z26XKW1Uax,0.693194,"{'product_uid': 168743, 'product_title': 'Lock...",168743,LockState Angle L Bracket for 600 lb.,LS-3320 angle l bracket for LS-600S. Used to m...,"[{'name': 'Bullet01', 'value': 'Satin finish',...",None,NaN
1,products-embeddings,0ZIVFo8B20Z26XKWTVM4,0.679624,"{'product_uid': 174846, 'product_title': 'Marq...",174846,Marquee Railing Black Left Multi-Angle Bracket...,"Designed with a beautiful hammered-metal look,...","[{'name': 'Accessory type', 'value': 'Bracket'...",None,NaN
2,products-embeddings,BJENFo8B20Z26XKWioml,0.678604,"{'product_uid': 165217, 'product_title': 'Vera...",165217,Veranda Vinyl Wicker Premier Rail Left-Right A...,The Veranda Vinyl White Left/Right Angle Brack...,"[{'name': 'Accessory type', 'value': 'Left/Rig...",45476ecfe43c98557ae68d2eec00a37d8f21c0fef3d913...,3.0
3,products-embeddings,S5IUFo8B20Z26XKW9Ep8,0.678497,"{'product_uid': 170398, 'product_title': 'Marq...",170398,Marquee Railing Black Right Multi-Angle Bracke...,"Designed with a beautiful hammered-metal look,...","[{'name': 'Accessory type', 'value': 'Bracket'...",None,NaN
4,products-embeddings,ZJISFo8B20Z26XKWogoJ,0.677744,"{'product_uid': 137713, 'product_title': 'Vera...",137713,Veranda 3-1/2 in. x 3-1/2 in. x 3 in. Vinyl Tr...,The Veranda Vinyl Stair Angle Brackets (4-Pack...,[],None,NaN


## Run the vector queries

In [16]:
# Filter down to queries that have results

relevant_queries = df_queries[df_queries['has_relevant_results']==1]

In [17]:
# Create query result dictionaries

filename = "query_runs/run_vector.json"

if os.path.isfile(filename):
    with open(filename, "r") as file:
        run_vu = json.load(file)
else:
    start_time = time.time()
    run_vu = {}

    for index,row in tqdm(relevant_queries.iterrows(), total=len(relevant_queries)):
        search_vector = row['query_embedding']
        search_query_id = row['query_id']

        results = query_elasticsearch_hybrid(es, index_name, search_vector=search_vector, num_results=10)
        hits = pd.DataFrame(results['hits']['hits'])

        query_doc_dict = {}
        for index, row in hits.iterrows():
            query_doc_dict[str(row['_source']['product_uid'])] = row['_score']
        
        run_vu[search_query_id] = query_doc_dict

    end_time = time.time() 

    with open(filename, "w") as file:
        json.dump(run_vu, file)

100%|██████████| 11795/11795 [05:06<00:00, 38.51it/s]


## Evaluate results

In [18]:
# Load ground truth
filename = "query_runs/qrel.json"

with open(filename, "r") as file:
    qrel = json.load(file)

In [19]:
# Place to store results and initialize an evaluator

ranking_results = []
measures = {'map_cut_10', 'ndcg_cut_10', 'recip_rank'} 
evaluator = pytrec_eval.RelevanceEvaluator(qrel, measures)

In [20]:
# Evaluate vector queries

results_df = pd.DataFrame(evaluator.evaluate(run_vu))
results_dict = {}
for measure in measures:
    results_dict['mean '+measure] = results_df.loc[measure].mean()
results_df = pd.DataFrame(results_dict, index=[0])

results_df['run_name'] = 'vectorsearch'
results_df['run_time'] = end_time - start_time
results_df['run_timestamp'] = current_timestamp

ranking_results.append(results_df)

In [21]:
# Put the results into a dataframe, add to previous results

ranking_results = pd.concat(ranking_results)

all_results = pd.read_csv('query_runs/query_results.csv')
all_results = pd.concat([all_results, ranking_results])
all_results

,mean ndcg_cut_10,mean map_cut_10,mean recip_rank,run_name,run_time,run_timestamp
0,0.170037,0.112637,0.261150,textsearch,178.504840,2024-05-06 20:30:09.220148
1,0.217595,0.149411,0.317710,textsearch_boosted,207.407446,2024-05-06 20:30:09.220148
0,0.245862,0.165978,0.342546,vectorsearch,306.276232,2024-05-06 20:39:12.634220


In [22]:
all_results.to_csv('query_runs/query_results.csv', index=False)

# Sample Queries

In [23]:
boosts = {"title_boost": 8,
          "description_boost": 2,
          "attributes_boost": 1}

In [24]:
sample_query_id = '083a28c9e216c858cafb3f3a08004fc9f9afa893b5a24612625487bb63d82e0e'

# A sample text query

search_text = df_queries[df_queries['query_id']==sample_query_id]['search_term'].values[0]
print(search_text)

results = query_elasticsearch_hybrid(es, 'products-embeddings', search_text=search_text, boost_values=boosts)

hits = pd.DataFrame(results['hits']['hits'])

real flame gel fuel


In [25]:

product_uids = []
product_titles = []
product_descriptions = []
product_attributes = []
query_id_list = []
relevances = []
for h in hits['_source']:
    product_uids.append(h['product_uid'])
    product_titles.append(h['product_title'])
    product_descriptions.append(h['product_description'])
    product_attributes.append(h['product_attributes'])
    
    if len(h['query_scores'])>0:
        query_id_list.append([x['query_id'] for x in h['query_scores']])
        relevances.append([x['relevance'] for x in h['query_scores']])
    else:
        query_id_list.append(None)
        relevances.append(None)

hits['product_uid'] = product_uids
hits['product_title'] = product_titles
hits['product_description'] = product_descriptions
hits['product_attribute'] = product_attributes
hits['query_id'] = query_id_list
hits['relevance'] = relevances

hits.head(10)


,_index,_id,_score,_source,product_uid,product_title,product_description,product_attribute,query_id,relevance
0,products-embeddings,p5IZFo8B20Z26XKW69A8,295.25946,"{'product_uid': 220338, 'product_title': 'Real...",220338,Real Flame Porter 50 in. Ventless Gel Fuel Fir...,The Porter Fireplace features distinct craftsm...,"[{'name': 'Area Heated (Sq. Ft.)', 'value': '1...",None,None
1,products-embeddings,spESFo8B20Z26XKWJ_0o,294.44052,"{'product_uid': 130661, 'product_title': 'Real...",130661,Real Flame Chateau 41 in. Ventless Gel Fuel Fi...,The Chateau Fireplace features the clean lines...,"[{'name': 'Area Heated (Sq. Ft.)', 'value': '1...",None,None
2,products-embeddings,uJENFo8B20Z26XKWyY5v,292.37097,"{'product_uid': 168245, 'product_title': 'Real...",168245,Real Flame Ashley 48 in. Gel Fuel Fireplace in...,Best-selling gel fireplace. The handsome pilla...,"[{'name': 'Area Heated (Sq. Ft.)', 'value': '2...",[c632e8a28835563a0de6fd31a562d80b4f7653a024b6c...,[2.0]
3,products-embeddings,m5EKFo8B20Z26XKWVD3C,290.45102,"{'product_uid': 127790, 'product_title': 'Real...",127790,Real Flame Ashley 48 in. Gel Fuel Fireplace in...,Best-selling fireplace. This handsome pillar w...,"[{'name': 'Area Heated (Sq. Ft.)', 'value': '2...",[2b77823854286393e74a6c020bcb4061ea0237d23d60a...,"[2.67, 2.67, 2.67]"
4,products-embeddings,d5EOFo8B20Z26XKWOZm3,290.45102,"{'product_uid': 174255, 'product_title': 'Real...",174255,Real Flame Ashley 48 in. Gel Fuel Fireplace in...,Best-selling fireplace. The handsome pillars w...,"[{'name': 'Area Heated (Sq. Ft.)', 'value': '2...",[c632e8a28835563a0de6fd31a562d80b4f7653a024b6c...,[2.33]
5,products-embeddings,MJIUFo8B20Z26XKW2EeR,286.56256,"{'product_uid': 168910, 'product_title': 'Real...",168910,Real Flame Chateau 41 in. Corner Ventless Gel ...,The Chateau Corner Fireplace features the clea...,"[{'name': 'Area Heated (Sq. Ft.)', 'value': '1...",None,None
6,products-embeddings,fJELFo8B20Z26XKWLVEe,286.56256,"{'product_uid': 137119, 'product_title': 'Real...",137119,Real Flame Chateau 41 in. Corner Ventless Gel ...,The Chateau Corner Fireplace features the clea...,"[{'name': 'Area Heated (Sq. Ft.)', 'value': '1...",[2b77823854286393e74a6c020bcb4061ea0237d23d60a...,"[2.67, 2.33]"
7,products-embeddings,NpEQFo8B20Z26XKWpdS1,280.71036,"{'product_uid': 104458, 'product_title': 'Real...",104458,Real Flame Silverton 48 in. Gel Fuel Fireplace...,Curl up by the comforting glow of this Real Fl...,[],None,None
8,products-embeddings,BJITFo8B20Z26XKWgyNz,279.90768,"{'product_uid': 151041, 'product_title': 'Real...",151041,Real Flame Chateau 41 in. Corner Ventless Gel ...,The Chateau Corner Fireplace features the clea...,"[{'name': 'Area Heated (Sq. Ft.)', 'value': '1...",None,None
9,products-embeddings,d5IYFo8B20Z26XKWl6-S,279.46896,"{'product_uid': 211843, 'product_title': 'Real...",211843,Real Flame 15 in. 2-Can Outdoor Gel Fuel Conve...,Convert your existing outdoor fire pit or fire...,"[{'name': 'Assembled Depth (in.)', 'value': '1...",None,None


In [26]:
# A sample vector query

search_vector = df_queries[df_queries['query_id']==sample_query_id]['query_embedding'].values[0]
results = query_elasticsearch_hybrid(es, 'products-embeddings', search_vector=search_vector)
hits = pd.DataFrame(results['hits']['hits'])

In [27]:

product_uids = []
product_titles = []
product_descriptions = []
product_attributes = []
query_id_list = []
relevances = []
for h in hits['_source']:
    product_uids.append(h['product_uid'])
    product_titles.append(h['product_title'])
    product_descriptions.append(h['product_description'])
    product_attributes.append(h['product_attributes'])
    
    if len(h['query_scores'])>0:
        query_id_list.append([x['query_id'] for x in h['query_scores']])
        relevances.append([x['relevance'] for x in h['query_scores']])
    else:
        query_id_list.append(None)
        relevances.append(None)

hits['product_uid'] = product_uids
hits['product_title'] = product_titles
hits['product_description'] = product_descriptions
hits['product_attribute'] = product_attributes
hits['query_id'] = query_id_list
hits['relevance'] = relevances

hits.head(10)


,_index,_id,_score,_source,product_uid,product_title,product_description,product_attribute,query_id,relevance
0,products-embeddings,EpEJFo8B20Z26XKWxS2Y,0.809501,"{'product_uid': 120490, 'product_title': 'Real...",120490,Real Flame 13 oz. 18.5 lb. Gel Fuel Cans (16-P...,Real Flame is the leading brand of gel fuel in...,[],[083a28c9e216c858cafb3f3a08004fc9f9afa893b5a24...,[3.0]
1,products-embeddings,95EMFo8B20Z26XKW0ncZ,0.804812,"{'product_uid': 156148, 'product_title': 'Real...",156148,Real Flame 13 oz. 24 lb. Gel Fuel Cans (24-Pack),Real Flame is the leading brand of gel fuel in...,[],[083a28c9e216c858cafb3f3a08004fc9f9afa893b5a24...,[3.0]
2,products-embeddings,fJENFo8B20Z26XKWG35R,0.803864,"{'product_uid': 159600, 'product_title': 'Real...",159600,Real Flame 13 oz. 15 lb. Gel Fuel Cans (12-Pack),Real Flame is the leading brand of gel fuel in...,[],[083a28c9e216c858cafb3f3a08004fc9f9afa893b5a24...,[3.0]
3,products-embeddings,55ITFo8B20Z26XKWcCBY,0.776211,"{'product_uid': 149940, 'product_title': 'Real...",149940,Real Flame 24 in. Oak Convert to Gel Fireplace...,Convert your existing gas or wood-burning fire...,[],None,None
4,products-embeddings,gZENFo8B20Z26XKWb4bS,0.775063,"{'product_uid': 163869, 'product_title': 'Real...",163869,Real Flame Fresno 72 in. Media Console Gel Fue...,Enjoy the beauty of a Real Flame fireplace. Bu...,[],[083a28c9e216c858cafb3f3a08004fc9f9afa893b5a24...,[2.67]
5,products-embeddings,NpEQFo8B20Z26XKWpdS1,0.769471,"{'product_uid': 104458, 'product_title': 'Real...",104458,Real Flame Silverton 48 in. Gel Fuel Fireplace...,Curl up by the comforting glow of this Real Fl...,[],None,None
6,products-embeddings,KpEKFo8B20Z26XKWs0UP,0.764870,"{'product_uid': 131293, 'product_title': 'Real...",131293,Real Flame Hawthorne 75 in. Media Console Gel ...,The Hawthorne Gel Fireplace features mission i...,[],[07006d6c8e46f85476905aec60a75ee91bbc86d05fed6...,[2.33]
7,products-embeddings,DZELFo8B20Z26XKWnl2A,0.763937,"{'product_uid': 142622, 'product_title': 'Real...",142622,Real Flame 18 in. Oak Convert to Gel Fireplace...,Convert your existing gas or wood-burning fire...,[],[083a28c9e216c858cafb3f3a08004fc9f9afa893b5a24...,[2.0]
8,products-embeddings,X5ITFo8B20Z26XKWayB4,0.763640,"{'product_uid': 149669, 'product_title': 'Real...",149669,Real Flame Chateau 41 in. Ventless Gel Fuel Fi...,The Chateau Fireplace features the clean lines...,[],None,None
9,products-embeddings,d5IYFo8B20Z26XKWl6-S,0.763134,"{'product_uid': 211843, 'product_title': 'Real...",211843,Real Flame 15 in. 2-Can Outdoor Gel Fuel Conve...,Convert your existing outdoor fire pit or fire...,"[{'name': 'Assembled Depth (in.)', 'value': '1...",None,None
